# Maximum Clique Experiments — Medium Graphs

In [ ]:
import os
import time
import importlib
import sys
import tracemalloc
import networkx as nx
import matplotlib.pyplot as plt

# Ensure algorithms folder is importable
if "algorithms" not in sys.path:
    sys.path.append("../algorithms")

def load_graph(path):
    return nx.read_adjlist(path, nodetype=int)

def visualize_graph_with_clique(G, clique_nodes, title=None):
    pos = nx.spring_layout(G, seed=42)
    clique_set = set(clique_nodes)
    node_colors = ["gold" if n in clique_set else "lightgray" for n in G.nodes()]
    node_sizes = [250 if n in clique_set else 80 for n in G.nodes()]

    plt.figure(figsize=(7, 5))
    nx.draw_networkx_edges(G, pos, alpha=0.4)
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes)
    if G.number_of_nodes() <= 60:
        nx.draw_networkx_labels(G, pos, font_size=8)
    plt.title(title or f"Graph (n={G.number_of_nodes()}, m={G.number_of_edges()}) — max clique size {len(clique_nodes)}")
    plt.axis("off")
    plt.show()

def run_algorithm(module_name, G):
    module = importlib.import_module(module_name)
    tracemalloc.start()
    start = time.perf_counter()
    clique = module.maximum_clique(G)
    end = time.perf_counter()
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return clique, end - start, peak / 1024


In [ ]:
graph_size = 'medium'
G = load_graph(f'../data/medium_graphs/temp.txt')
print(G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges')

In [ ]:
results = {}

In [ ]:
import bron_kerbosch
clique, runtime, peak = run_algorithm("bron_kerbosch", G)
results["bron_kerbosch"] = {"runtime": runtime, "peak": peak, "size": len(clique)}
print(f"bron_kerbosch: runtime={runtime:.4f}s, peak={peak:.1f} KiB, size={len(clique)}")
visualize_graph_with_clique(G, clique, title="bron_kerbosch")


In [ ]:
import greedy
clique, runtime, peak = run_algorithm("greedy", G)
results["greedy"] = {"runtime": runtime, "peak": peak, "size": len(clique)}
print(f"greedy: runtime={runtime:.4f}s, peak={peak:.1f} KiB, size={len(clique)}")
visualize_graph_with_clique(G, clique, title="greedy")


In [ ]:
import local_search
clique, runtime, peak = run_algorithm("local_search", G)
results["local_search"] = {"runtime": runtime, "peak": peak, "size": len(clique)}
print(f"local_search: runtime={runtime:.4f}s, peak={peak:.1f} KiB, size={len(clique)}")
visualize_graph_with_clique(G, clique, title="local_search")


In [ ]:
import randomized
clique, runtime, peak = run_algorithm("randomized", G)
results["randomized"] = {"runtime": runtime, "peak": peak, "size": len(clique)}
print(f"randomized: runtime={runtime:.4f}s, peak={peak:.1f} KiB, size={len(clique)}")
visualize_graph_with_clique(G, clique, title="randomized")


In [ ]:
import simulated_annealing
clique, runtime, peak = run_algorithm("simulated_annealing", G)
results["simulated_annealing"] = {"runtime": runtime, "peak": peak, "size": len(clique)}
print(f"simulated_annealing: runtime={runtime:.4f}s, peak={peak:.1f} KiB, size={len(clique)}")
visualize_graph_with_clique(G, clique, title="simulated_annealing")


In [ ]:
import matplotlib.pyplot as plt

algs = list(results.keys())
runtimes = [results[a]['runtime'] for a in algs]
peaks = [results[a]['peak'] for a in algs]

plt.figure(figsize=(8, 4))
plt.bar(algs, runtimes)
plt.ylabel("Runtime (s)")
plt.title(f"Algorithm runtimes on {graph_size} graph")
plt.xticks(rotation=30)
plt.show()

plt.figure(figsize=(8, 4))
plt.bar(algs, peaks)
plt.ylabel("Peak memory (KiB)")
plt.title(f"Algorithm memory on {graph_size} graph")
plt.xticks(rotation=30)
plt.show()
